In [30]:
import pandas as pd # needed for most operation
import numpy as np # needed for some array operations
from sqlalchemy import create_engine # needed for DB connection.
import matplotlib.pyplot as plt
import csv
from datetime import timedelta
from sqlalchemy import text

In [31]:
with open('SuperstoreSales_main.csv','r',encoding='ISO-8859-1') as inf , open('output.csv', 'w',encoding='ISO-8859-1') as of:
    r = csv.reader(inf, delimiter=';')
    w = csv.writer(of, delimiter=';')
    for line in r:
        trim = (field.strip() for field in line )
        w.writerow(trim)

# read csv files
sales = pd.read_csv('output.csv', encoding='ISO-8859-1',delimiter=';')
product = pd.read_csv('output.csv', encoding='ISO-8859-1',delimiter=';')
customer = pd.read_csv('output.csv', encoding='ISO-8859-1',delimiter=';')
returnstatus = pd.read_csv('SuperstoreSales_returns.csv', encoding='ISO-8859-1',delimiter=';')

product = product[['Product Category', 'Product Sub-Category', 'Product Name']]\
    .drop_duplicates(subset=['Product Name','Product Category','Product Sub-Category'] , ignore_index=True, keep='last')
product = product[['Product Name', 'Product Category','Product Sub-Category']].rename(columns={'Product Name': 'name', 'Product Category': 'category', 'Product Sub-Category':'subcategory'})
product['productid'] = product.reset_index().index
product = product[['productid','name','category','subcategory']]
product.to_csv('Product.csv', index=False, encoding='ISO-8859-1', mode='w')

customer = customer[['Customer Name', 'Province', 'Region', 'Customer Segment']]\
    .drop_duplicates(subset=['Customer Name', 'Province', 'Region', 'Customer Segment'] , ignore_index=True, keep='last')\
    .rename(columns={'Customer Name': 'customer_name', 'Province': 'province', 'Region': 'region', 'Customer Segment': 'segment'})
customer['customerid'] = customer.reset_index().index
customer = customer[['customerid','customer_name','province', 'region', 'segment']]
customer.to_csv('Customer.csv', index=False, encoding='ISO-8859-1', mode='w')

sales = sales[['Order Date', 'Order Quantity', 'Sales','Profit', 'Unit Price', 'Shipping Cost','Product Name',
                'Product Category','Customer Name', 'Province', 'Region', 'Customer Segment','Order ID','Ship Date']]
sales = pd.merge(sales, product, how='outer',left_on = ['Product Name','Product Category'],right_on = ['name','category'])\
    .drop(columns=['Product Name', 'Product Category', 'name','category'], errors='ignore')
sales = pd.merge(sales, customer, how='outer',left_on = ['Customer Name','Province', 'Region', 'Customer Segment'],\
                 right_on = ['customer_name','province', 'region', 'segment'])\
    .drop(columns=['Customer Name','Province', 'Region', 'Customer Segment', 'customer_name','province', 'region', 'segment'], errors='ignore')

sales['orderdate'] = pd.to_datetime(sales['Order Date'], format = "%d/%m/%y")
sales = sales.drop(columns=['Order Date'], errors='ignore')
sales['returnstatusid'] = 0.0
returnstatus = returnstatus.rename(columns={'Status': 'returnstatusid'}).replace('Returned', value=1)
sales = pd.merge(sales, returnstatus, how='left', on = ['Order ID'], suffixes=('A','B'))
sales['returnstatusidB'] = sales['returnstatusidB'].fillna(0)
sales = sales.rename(columns={'returnstatusidB': 'returnstatusid'}).drop(columns={'returnstatusidA'})
sales = sales.astype({'returnstatusid':'int'})
sales['Ship Date'] = pd.to_datetime(sales['Ship Date'], format = "%d/%m/%y")
expected_late_date = sales['orderdate'] + pd.Timedelta(days=2)
sales['late'] = np.where(sales['Ship Date'] < expected_late_date, 'NotLate', 'Late')

sales = sales.drop(columns=['Ship Date'], errors='ignore')\
    .rename(columns={'Shipping Cost': 'shippingcost', 'Unit Price': 'unitprice', 'Profit': 'profit', 'Sales': 'sales', 'Order Quantity': 'orderquantity',"Order ID": "orderId"})
sales = sales[['orderId','productid', 'customerid', 'orderdate', 'returnstatusid', 'late', 'sales', 'orderquantity', 'unitprice', 'profit', 'shippingcost']]

returnStatus = pd.DataFrame({'returnstatusid': [0, 1], 'returnvalue': ['NotReturned', 'Returned']})
returnStatus.to_csv('ReturnStatus.csv', index=False, encoding='ISO-8859-1', mode='w')

sales.to_csv('Sales.csv', index=False, encoding='ISO-8859-1', mode='w')

print('Unique Orders: '+str(sales['orderId'].nunique()))
print('Unique returned Orders:'+str(returnstatus['Order ID'].nunique()))

driver='postgresql'
username='dab_ds23241a_237'
dbname=username # it is the same as the username
password='yaiy2gzjumSdTfVA'
server='bronto.ewi.utwente.nl'
port='5432'

# Creating the connetcion pool for SQL
engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')
product.to_sql('Product', engine,schema='ass3', index=False, if_exists='replace')
sales.drop(columns=['orderId'], errors='ignore').to_sql('Sales', engine,schema='ass3',index=False, if_exists='replace')
customer.to_sql('Customer', engine,schema='ass3',index=False, if_exists='replace')
returnStatus.to_sql('ReturnStatus', engine,schema='ass3',index=False, if_exists='replace')
with engine.connect() as conn:
    result = conn.execute(text("""SELECT table_name FROM information_schema.tables
where table_schema='ass3'"""))

pd.read_sql_table('Product', engine, schema='ass3').info()
pd.read_sql_table('Customer', engine, schema='ass3').info()
pd.read_sql_table('Sales', engine, schema='ass3').info()
pd.read_sql_table('ReturnStatus', engine, schema='ass3').info()


Unique Orders: 5496
Unique returned Orders:572
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1263 entries, 0 to 1262
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   productid    1263 non-null   int64 
 1   name         1263 non-null   object
 2   category     1263 non-null   object
 3   subcategory  1263 non-null   object
dtypes: int64(1), object(3)
memory usage: 39.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832 entries, 0 to 1831
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerid     1832 non-null   int64 
 1   customer_name  1832 non-null   object
 2   province       1832 non-null   object
 3   region         1832 non-null   object
 4   segment        1832 non-null   object
dtypes: int64(1), object(4)
memory usage: 71.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8399 entries, 0 to 8398
Data columns (tota